In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm
from torchsummary import summary

In [ ]:
class Net(nn.Module): # image siE in mnist is 1x28x28
                                           


    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) 
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) # 1 convolutional block
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [ ]:
# Checking that cuda is available or not
torch.cuda.is_available()

True

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=1.99245023727417 batch_id=468: 100%|██████████| 469/469 [00:48<00:00,  9.57it/s]



Test set: Average loss: 1.9687, Accuracy: 2790/10000 (28%)



Structure 2

In [ ]:
class Net2(nn.Module): # image siE in mnist is 1x28x28
    def __init__(self):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(1, 1024, 3, padding=1)  
        self.conv2 = nn.Conv2d(1024, 512, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(512, 256, 3, padding=1)
        self.conv4 = nn.Conv2d(256, 128, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(128, 64, 3)
        self.conv6 = nn.Conv2d(64, 32, 3)
        self.conv7 = nn.Conv2d(32, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) 
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [ ]:
use_cuda = torch.cuda.is_available()
device2 = torch.device("cuda" if use_cuda else "cpu")
model2 = Net2().to(device2)
summary(model2, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 1024, 28, 28]          10,240
            Conv2d-2          [-1, 512, 28, 28]       4,719,104
         MaxPool2d-3          [-1, 512, 14, 14]               0
            Conv2d-4          [-1, 256, 14, 14]       1,179,904
            Conv2d-5          [-1, 128, 14, 14]         295,040
         MaxPool2d-6            [-1, 128, 7, 7]               0
            Conv2d-7             [-1, 64, 5, 5]          73,792
            Conv2d-8             [-1, 32, 3, 3]          18,464
            Conv2d-9             [-1, 10, 1, 1]           2,890
Total params: 6,299,434
Trainable params: 6,299,434
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 10.59
Params size (MB): 24.03
Estimated Total Size (MB): 34.62
------------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
model2 = Net2().to(device2)
optimizer = optim.SGD(model2.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model2, device2, train_loader, optimizer, epoch)
    test(model2, device2, test_loader)

loss=1.567403793334961 batch_id=468: 100%|██████████| 469/469 [03:41<00:00,  2.12it/s]



Test set: Average loss: 1.4030, Accuracy: 4093/10000 (41%)



Structure 3

In [ ]:
class Net3(nn.Module): # image siE in mnist is 1x28x28

    def __init__(self):
        super(Net3, self).__init__()
        self.conv0 = nn.Conv2d(1, 32, 3, padding=1) #28x28x1 -> 28x28x32 
        self.conv1 = nn.Conv2d(32, 64, 3, padding=1) #28x28x32 -> 28x28x64
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1) #28x28x64 -> 28x28x128
        self.pool1 = nn.MaxPool2d(2, 2) #28x28x128 -> 14x14x128
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1) #14x14x128 -> 14x14x256
        self.conv4 = nn.Conv2d(256, 512, 3, padding=1) #14x14x256 -> 14x14x512
        self.conv5 = nn.Conv2d(512, 1024, 3, padding=1) #14x14x512 -> 14x14x1024
        self.pool2 = nn.MaxPool2d(2, 2) #14x14x1024 -> 7x7x1024
        self.conv6 = nn.Conv2d(1024, 2048, 3) #7x7x1024 -> 5x5x2048
        self.conv7 = nn.Conv2d(2048, 4096, 3) #5x5x2048 -> 3x3x4096
        self.conv8 = nn.Conv2d(4096, 10, 3) #3x3x4096 -> 1x1x10

       
    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(F.relu(self.conv0(x))))))) 
        x = self.pool2(F.relu(self.conv5(F.relu(self.conv4(F.relu(self.conv3(x))))))) 
        x = F.relu(self.conv7(F.relu(self.conv6(x))))
        x = F.relu(self.conv8(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [ ]:
use_cuda = torch.cuda.is_available()
device3 = torch.device("cuda" if use_cuda else "cpu")
model3 = Net3().to(device3)
summary(model3, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
            Conv2d-3          [-1, 128, 28, 28]          73,856
         MaxPool2d-4          [-1, 128, 14, 14]               0
            Conv2d-5          [-1, 256, 14, 14]         295,168
            Conv2d-6          [-1, 512, 14, 14]       1,180,160
            Conv2d-7         [-1, 1024, 14, 14]       4,719,616
         MaxPool2d-8           [-1, 1024, 7, 7]               0
            Conv2d-9           [-1, 2048, 5, 5]      18,876,416
           Conv2d-10           [-1, 4096, 3, 3]      75,501,568
           Conv2d-11             [-1, 10, 1, 1]         368,650
Total params: 101,034,250
Trainable params: 101,034,250
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
model3 = Net3().to(device3)
optimizer = optim.SGD(model3.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model3, device3, train_loader, optimizer, epoch)
    test(model3, device3, test_loader)

loss=1.4544988870620728 batch_id=468: 100%|██████████| 469/469 [05:56<00:00,  1.31it/s]



Test set: Average loss: 1.4865, Accuracy: 4785/10000 (48%)



4

In [ ]:
class Net4(nn.Module): # image siE in mnist is 1x28x28
    def __init__(self):
        super(Net4, self).__init__()
        self.conv1 = nn.Conv2d(1, 1024, 5, padding=2)  
        self.conv2 = nn.Conv2d(1024, 512, 5, padding=2)
        self.conv3 = nn.Conv2d(512, 256, 5, padding=2)
        self.conv4 = nn.Conv2d(256, 128, 5, padding=2)
        self.conv5 = nn.Conv2d(128, 64, 5)
        self.conv6 = nn.Conv2d(64, 32, 5)
        self.conv7 = nn.Conv2d(32, 10, 5)
        self.conv8 = nn.Conv2d(10, 10, 5)
        self.conv9 = nn.Conv2d(10, 10, 5)
        self.conv10 = nn.Conv2d(10, 10, 5)
        self.conv11 = nn.Conv2d(10, 10, 4)

    def forward(self, x):
        x = F.relu(self.conv2(F.relu(self.conv1(x)))) 
        x = F.relu(self.conv4(F.relu(self.conv3(x))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv8(F.relu(self.conv7(x))))
        x = F.relu(self.conv10(F.relu(self.conv9(x))))
        x = F.relu(self.conv11(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [ ]:
use_cuda = torch.cuda.is_available()
device4 = torch.device("cuda" if use_cuda else "cpu")
model4 = Net4().to(device4)
summary(model4, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 1024, 28, 28]          26,624
            Conv2d-2          [-1, 512, 28, 28]      13,107,712
            Conv2d-3          [-1, 256, 28, 28]       3,277,056
            Conv2d-4          [-1, 128, 28, 28]         819,328
            Conv2d-5           [-1, 64, 24, 24]         204,864
            Conv2d-6           [-1, 32, 20, 20]          51,232
            Conv2d-7           [-1, 10, 16, 16]           8,010
            Conv2d-8           [-1, 10, 12, 12]           2,510
            Conv2d-9             [-1, 10, 8, 8]           2,510
           Conv2d-10             [-1, 10, 4, 4]           2,510
           Conv2d-11             [-1, 10, 1, 1]           1,610
Total params: 17,503,966
Trainable params: 17,503,966
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Fo

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
model4 = Net4().to(device4)
optimizer = optim.SGD(model3.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model4, device4, train_loader, optimizer, epoch)
    test(model4, device4, test_loader)

loss=2.2988624572753906 batch_id=468: 100%|██████████| 469/469 [09:19<00:00,  1.19s/it]



Test set: Average loss: 2.3019, Accuracy: 1135/10000 (11%)

